<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c3e031bf074e2018593d6fc99d2a1d1874ef7e3e31cd2ad6b29166cbd4ea8fc9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-14 15:18:33
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -36.68 K (-0.26%)
Current PnL: -28.86 L (-18.4%)
CY Booked + Current PnL: -14.57 L (-9.29%)
-------------------
Total profit:  1.62 L
Total loss:  -30.47 L
-------------------
Total Booked + Current PnL: 12.41 L (9.56%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.19%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.31 L (67.26%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.16%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.89,15.23,5.36,21.40,16393.0,40412.0,305838.0,6.96,60.0,X-LC,6.35,14.0,2.47,2.18,22.52,XY25,NTT,FMCG
17,COALINDIA,484.83,0.72,6.23,12.23,19.23,19548.0,9380.0,159840.0,24.27,70.0,L-LC,8.64,182.0,0.48,1.14,28.84,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-0.35,-22.99,29.97,0.09,23260.0,-23166.0,77611.0,80.15,31.0,M-SC,3.72,253.0,-1.00,0.55,1.96,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.88,-14.75,17.32,0.02,25774.0,-25747.0,148812.0,127.12,52.0,M-SC,8.79,240.0,-1.00,1.06,29.44,OX40N,NTT,PAINTS
49,MASFIN,398.61,-0.38,-4.75,28.13,22.05,26254.0,-4650.0,93330.0,-17.80,45.0,H-SC,5.59,168.0,-0.18,0.67,35.66,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,4.84,-28.81,58.60,12.90,91148.0,-62954.0,155542.0,129.57,60.0,H-SC,9.40,173.0,-0.69,1.11,29.33,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,3.84,-30.19,90.37,32.89,48706.0,-23310.0,53896.0,-765.75,73.0,L-MC,8.62,259.0,-0.48,0.38,48.43,XR,NTT,BANKS
74,TITAGARH,1548.00,3.16,-23.07,89.45,45.74,166644.0,-55878.0,186299.0,2.48,45.0,H-SC,3.03,175.0,-0.34,1.33,21.17,XY24,NTT,ENGINEERING
12,BERGEPAINT,680.00,2.13,-7.23,30.58,21.14,64494.0,-16435.0,210904.0,-13.78,41.0,X-MC,2.90,75.0,-0.25,1.50,14.26,XY24,NTT,PAINTS
4,ALKYLAMINE,4546.37,1.97,-30.04,176.76,93.63,124856.0,-30327.0,70636.0,-25.06,55.0,H-SC,16.60,143.0,-0.24,0.50,6.79,SR,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,-5.00,-20.68,66.46,32.03,69495.0,-27269.0,104566.0,-14.19,53.0,H-SC,6.66,157.0,-0.39,0.75,17.12,OX40N,NTT,IT
71,TANLA,1943.92,-2.73,-45.25,301.26,119.69,446594.0,-122519.0,148242.0,-44.56,29.0,H-SC,11.74,150.0,-0.27,1.06,17.11,AR,ATH,IT
5,ASIANPAINT,3460.26,-2.57,-1.71,23.05,20.95,57041.0,-4303.0,247465.0,-2.74,51.0,X-LC,12.74,36.0,-0.08,1.77,33.32,X40,ATH,PAINTS
44,JCHAC,2282.00,-2.33,-37.18,59.17,-0.01,47506.0,-47518.0,80287.0,16334.78,40.0,M-SC,13.40,235.0,-1.00,0.57,2.76,OX40N,BTT,AC
73,TCS,4389.97,-2.31,-12.22,37.51,20.71,124541.0,-46217.0,332020.0,-24.27,46.0,X-LC,2.12,3.0,-0.37,2.37,10.94,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.72,0.04,20.69,20.74,42053.0,82.0,203252.0,31.15,55.0,M-MC,5.23,189.0,0.0,1.45,10.66,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.82,-1.75,111.62,107.91,161912.0,-2588.0,145056.0,-20.15,53.0,M-SC,10.09,220.0,-0.02,1.03,8.16,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.88,-14.75,17.32,0.02,25774.0,-25747.0,148812.0,127.12,52.0,M-SC,8.79,240.0,-1.00,1.06,29.44,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-5.00,-20.68,66.46,32.03,69495.0,-27269.0,104566.0,-14.19,53.0,H-SC,6.66,157.0,-0.39,0.75,17.12,OX40N,NTT,IT
47,KANSAINER,340.00,1.87,-21.04,43.70,13.47,93055.0,-56727.0,212940.0,-67.53,54.0,H-SC,4.06,159.0,-0.61,1.52,8.27,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.35,-22.99,29.97,0.09,23260.0,-23166.0,77611.0,80.15,31.0,M-SC,3.72,253.0,-1.00,0.55,1.96,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.72,0.04,20.69,20.74,42053.0,82.0,203252.0,31.15,55.0,M-MC,5.23,189.0,0.00,1.45,10.66,XY25,NTT,FINANCE
17,COALINDIA,484.83,0.72,6.23,12.23,19.23,19548.0,9380.0,159840.0,24.27,70.0,L-LC,8.64,182.0,0.48,1.14,28.84,XY25,ATH,MINING
84,WIPRO,420.00,-1.59,6.64,61.54,72.27,111203.0,11252.0,180700.0,-8.35,47.0,M-LC,6.27,101.0,0.10,1.29,13.23,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.59,6.64,61.54,72.27,111203.0,11252.0,180700.0,-8.35,47.0,M-LC,6.27,101.0,0.10,1.29,13.23,XR,NTT,IT
25,FINCABLES,1641.55,1.82,-1.75,111.62,107.91,161912.0,-2588.0,145056.0,-20.15,53.0,M-SC,10.09,220.0,-0.02,1.03,8.16,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.90,-9.08,127.38,106.72,142834.0,-11204.0,112132.0,-55.01,32.0,H-SC,8.06,138.0,-0.08,0.80,12.17,AR,ATH,MISC
37,IEX,219.00,-1.39,-5.43,57.16,48.63,108088.0,-10856.0,189098.0,-37.19,48.0,H-SC,13.21,137.0,-0.10,1.35,5.56,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-1.87,-6.80,45.52,35.63,88915.0,-14243.0,195331.0,-19.84,33.0,H-MC,3.59,119.0,-0.16,1.39,8.74,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.03,-17.94,35.01,10.78,69039.0,-43121.0,197197.0,-50.96,13.0,X-LC,8.48,1.0,-0.62,1.41,0.03,X40,NTT,FMCG
7,AWL,485.00,-0.95,-29.45,122.79,57.19,282537.0,-96029.0,230098.0,-66.69,20.0,X-MC,10.48,58.0,-0.34,1.64,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-1.30,-23.10,70.68,31.26,125437.0,-53296.0,177472.0,-64.05,25.0,X-MC,20.79,56.0,-0.42,1.27,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.17,-41.53,131.24,35.20,98733.0,-53439.0,75231.0,-1.37,26.0,X-SC,17.58,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
52,PAGEIND,51605.08,-0.92,-13.12,50.34,30.62,103675.0,-31104.0,205950.0,-38.20,29.0,X-MC,16.98,57.0,-0.30,1.47,0.13,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JIOFIN,387.00,0.83,-7.70,34.87,24.48,86051.0,-20588.0,246777.0,-9.21,39.0,X-LC,1.64,37.0,-0.24,1.76,43.04,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,1.30,-1.81,23.00,20.77,52876.0,-4249.0,229895.0,-10.33,35.0,X-MC,1.84,70.0,-0.08,1.64,4.98,X40N,NTT,BREWERIES
73,TCS,4389.97,-2.31,-12.22,37.51,20.71,124541.0,-46217.0,332020.0,-24.27,46.0,X-LC,2.12,3.0,-0.37,2.37,10.94,X40,ATH,IT
20,DABUR,735.00,-1.60,0.43,43.08,43.70,107332.0,1077.0,249145.0,-7.78,54.0,X-MC,2.16,73.0,0.01,1.78,15.55,XY24,BTT,FMCG
65,SIEMENS,4671.50,0.65,-20.29,57.45,25.51,85224.0,-37750.0,148345.0,-8.99,37.0,H-LC,2.36,50.0,-0.44,1.06,9.32,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.95,-29.45,122.79,57.19,282537.0,-96029.0,230098.0,-66.69,20.0,X-MC,10.48,58.0,-0.34,1.64,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.17,-41.53,131.24,35.20,98733.0,-53439.0,75231.0,-1.37,26.0,X-SC,17.58,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.30,-23.10,70.68,31.26,125437.0,-53296.0,177472.0,-64.05,25.0,X-MC,20.79,56.0,-0.42,1.27,0.0,X40N,ATH,IT
53,PGHH,17907.65,-0.34,-7.52,45.11,34.19,89072.0,-16064.0,197456.0,-34.93,30.0,X-MC,4.36,60.0,-0.18,1.41,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-0.62,-49.08,199.08,52.31,147164.0,-71238.0,73922.0,-45.79,36.0,X-SC,7.91,92.0,-0.48,0.53,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.75,-28.83,100.71,42.84,46592.0,-18742.0,46264.0,-53.17,47.0,X-SC,25.32,83.0,-0.40,0.33,4.67,XY24,NTT,MISC
58,RELAXO,1176.00,-0.62,-49.08,199.08,52.31,147164.0,-71238.0,73922.0,-45.79,36.0,X-SC,7.91,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.17,-41.53,131.24,35.20,98733.0,-53439.0,75231.0,-1.37,26.0,X-SC,17.58,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-0.76,-3.19,28.17,24.08,34620.0,-4054.0,122896.0,-11.10,43.0,X-SC,5.88,91.0,-0.12,0.88,15.80,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.74,-17.69,74.12,43.32,99365.0,-28812.0,134060.0,-29.29,45.0,X-SC,8.15,90.0,-0.29,0.96,2.93,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.03,-17.94,35.01,10.78,69039.0,-43121.0,197197.0,-50.96,13.0,X-LC,8.48,1.0,-0.62,1.41,0.03,X40,NTT,FMCG
75,TMPV,600.00,-0.06,-28.36,71.65,22.98,115960.0,-64056.0,161842.0,-25.52,37.0,X-LC,3.75,2.0,-0.55,1.15,1.79,XY24,NTT,AUTO
73,TCS,4389.97,-2.31,-12.22,37.51,20.71,124541.0,-46217.0,332020.0,-24.27,46.0,X-LC,2.12,3.0,-0.37,2.37,10.94,X40,ATH,IT
81,VBL,671.64,0.18,1.42,33.77,35.67,108179.0,4498.0,320340.0,-10.61,61.0,X-LC,2.61,5.0,0.04,2.28,15.48,X40N,ATH,FMCG
41,INFY,1972.00,-0.25,8.58,23.64,34.24,80313.0,26847.0,339735.0,-15.29,46.0,X-LC,7.56,6.0,0.33,2.42,17.57,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,3.84,-30.19,90.37,32.89,48706.0,-23310.0,53896.0,-765.75,73.0,L-MC,8.62,259.0,-0.48,0.38,48.43,XR,NTT,BANKS
13,BSOFT,831.70,-0.95,-23.66,97.53,50.79,104716.0,-33285.0,107368.0,-2.20,48.0,H-SC,10.19,171.0,-0.32,0.77,25.01,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.88,-14.75,17.32,0.02,25774.0,-25747.0,148812.0,127.12,52.0,M-SC,8.79,240.0,-1.00,1.06,29.44,OX40N,NTT,PAINTS
17,COALINDIA,484.83,0.72,6.23,12.23,19.23,19548.0,9380.0,159840.0,24.27,70.0,L-LC,8.64,182.0,0.48,1.14,28.84,XY25,ATH,MINING
36,ICICIPRULI,790.00,-1.97,10.54,18.26,30.73,39764.0,20763.0,217768.0,-15.30,55.0,X-MC,7.35,79.0,0.52,1.55,24.65,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,4.84,-28.81,58.60,12.90,91148.0,-62954.0,155542.0,129.57,60.0,H-SC,9.40,173.0,-0.69,1.11,29.33,OX40N,NTT,FINANCE
17,COALINDIA,484.83,0.72,6.23,12.23,19.23,19548.0,9380.0,159840.0,24.27,70.0,L-LC,8.64,182.0,0.48,1.14,28.84,XY25,ATH,MINING
40,INDUSINDBK,1800.00,3.84,-30.19,90.37,32.89,48706.0,-23310.0,53896.0,-765.75,73.0,L-MC,8.62,259.0,-0.48,0.38,48.43,XR,NTT,BANKS
72,TATAELXSI,9161.00,-5.00,-20.68,66.46,32.03,69495.0,-27269.0,104566.0,-14.19,53.0,H-SC,6.66,157.0,-0.39,0.75,17.12,OX40N,NTT,IT
57,RECLTD,446.00,0.72,0.04,20.69,20.74,42053.0,82.0,203252.0,31.15,55.0,M-MC,5.23,189.0,0.00,1.45,10.66,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.56
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.75
MC    30.60
LC    26.66
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.39
X40      23.94
X40N     12.83
XY25     10.84
XR        8.94
AR        8.44
OX40N     7.67
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.23
X-LC    23.17
H-SC    22.90
M-SC    11.20
X-SC     7.89
H-MC     4.54
M-MC     1.45
M-LC     1.29
L-LC     1.14
H-LC     1.06
L-SC     0.76
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.23,-9.04,44.82
IT,12.75,-19.75,80.15
FINANCE,11.13,-14.28,62.25
MISC,7.03,-31.76,85.66
ELECTRICAL,5.86,-14.33,56.21
PAINTS,5.85,-12.58,29.31
INSURANCE,4.69,-3.15,38.45
PHARMA,3.94,-6.19,39.62
BANKS,2.80,-36.39,119.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3269787.0,21
AR,1389750.0,10
XR,1238790.0,12
X40,1164702.0,15
X40N,944881.0,9
OX40N,716076.0,10
XY25,422254.0,7
SR,285049.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3586303.0,24
X-MC,1635566.0,16
M-SC,1486531.0,15
X-LC,975077.0,13
X-SC,815075.0,8
H-MC,442370.0,3
L-SC,183633.0,2
M-LC,111203.0,1
H-LC,85224.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1252346.0      6
           AR         939402.0      5
M-SC       XY24       852805.0      6
H-SC       XR         798324.0      7
X-MC       X40        537088.0      7
           XY24       454363.0      3
X-LC       X40        433872.0      6
X-MC       X40N       420626.0      4
X-SC       X40N       322225.0      3
H-SC       OX40N      311182.0      4
M-SC       OX40N      311097.0      5
X-SC       XY24       299108.0      3
H-SC       SR         285049.0      2
H-MC       AR         233216.0      2
X-MC       XY25       223489.0      2
H-MC       XY24       209154.0      1
X-LC       X40N       202030.0      2
           XY24       202011.0      2
X-SC       X40        193742.0      2
M-SC       XR         190721.0      2
X-LC       XY25       137164.0      3
M-SC       AR         131908.0      2
M-LC       XR         111203.0      1
L-SC       OX40N       93797.0      1
           XR          89836.0      1
H-LC       AR          85224.0      1
L-MC       XR          48706.0      1
M-MC       XY25        42053.0      1
L-LC       XY25        19548.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
